In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import GlobalAveragePooling2D, Activation, Dropout, Flatten, Dense
from keras.applications import ResNet50
from keras import optimizers
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Model

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

import pandas as pd
from tqdm import tqdm
import gc
import cv2 as cv

/home/faisal/anaconda3/envs/ai/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import numpy as np
import tensorflow as tf
import random as rn

# The below is necessary in Python 3.2.3 onwards to
# have reproducible behavior for certain hash-based operations.
# See these references for further details:
# https://docs.python.org/3.4/using/cmdline.html#envvar-PYTHONHASHSEED
# https://github.com/keras-team/keras/issues/2280#issuecomment-306959926

import os
os.environ['PYTHONHASHSEED'] = '0'
# os.environ['CUDA_VISIBLE_DEVICES'] = '0'

# The below is necessary for starting Numpy generated random numbers
# in a well-defined initial state.

np.random.seed(42)

# The below is necessary for starting core Python generated random numbers
# in a well-defined state.

rn.seed(12345)

# Force TensorFlow to use single thread.
# Multiple threads are a potential source of
# non-reproducible results.
# For further details, see: https://stackoverflow.com/questions/42022950/which-seeds-have-to-be-set-where-to-realize-100-reproducibility-of-training-res

session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)

from keras import backend as K

# The below tf.set_random_seed() will make random number generation
# in the TensorFlow backend have a well-defined initial state.
# For further details, see: https://www.tensorflow.org/api_docs/python/tf/set_random_seed

tf.set_random_seed(1234)

sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)


In [19]:
img_width, img_height = (224, 224)

train_data_dir = './data/train_img/'
test_data_dir = './data/test_img/'
batch_size = 64

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)


In [20]:
resnet_model = ResNet50(include_top=False, input_shape=input_shape)

In [5]:
def get_model():
# add a global spatial average pooling layer
    x = resnet_model.output
    x = GlobalAveragePooling2D()(x)
    predictions = Dense(85, activation='sigmoid')(x)
    model = Model(inputs=resnet_model.input, outputs=predictions)
    
    return model

In [6]:
model = get_model()

We'll start with training the head(last layer) only as that layer is initialized randomaly and we don't want to affect the other layers weights as while backpropogation. 

In [7]:
#train only last layer
for layer in model.layers[:-1]:
    layer.trainable = False

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 112, 112, 64) 0           bn_conv1[0][0]                   
__________________________________________________________________________________________________
max_poolin

In [28]:
train_df = pd.read_csv('./data/train.csv')
test_df = pd.read_csv('./data/test.csv')
attributes = pd.read_csv('./data/attributes.txt', delimiter='\t', header=None, index_col=0)
classes = pd.read_csv('./data/classes.txt', delimiter='\t', header=None, index_col=0)

train_df['Image_name'] = train_df['Image_name'].map(lambda x: './data/train_img/' + x)

train = shuffle(train)

In [29]:
train_split_df, valid_split_df = train_test_split(train_df, random_state = 222, test_size = 3000)


In [30]:
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(width_shift_range=0.2, 
                                   height_shift_range=0.2,
                                   rescale=1. / 255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True, 
                                   rotation_range = 20,
                                   channel_shift_range=0.2
                                   )

val_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)

In [31]:
def flow_from_dataframe(img_data_gen, in_df, path_col, y_col, seed = None, **dflow_args):
    base_dir = os.path.dirname(in_df[path_col].values[0])    
    print('## Ignore next message from keras, values are replaced anyways')
    df_gen = img_data_gen.flow_from_directory(base_dir, class_mode = 'sparse',seed = seed,**dflow_args)
    df_gen.filenames = in_df[path_col].values
    df_gen.classes = in_df.loc[:, in_df.columns != path_col].values
    df_gen.samples = in_df.shape[0]
    df_gen.n = in_df.shape[0]
    df_gen._set_index_array()
    df_gen.directory = '' # since we have the full path
    print('Reinserting dataframe: {} images'.format(in_df.shape[0]))
    
    return df_gen

def gen_func(in_df, rgb_gen, image_size = (224, 224), batch_size = 8, seed = None):
    if seed is None:
        seed = np.random.choice(range(1000))
        
    rgb_gen = flow_from_dataframe(rgb_gen, 
                                      in_df, 
                                      path_col = 'Image_name',
                                      y_col = 'id', 
                                      color_mode = 'rgb',
                                      target_size = image_size,
                                      batch_size = batch_size,
                                      seed = seed)
    for x, i in rgb_gen:
        idx = (rgb_gen.batch_index - 1) * rgb_gen.batch_size
        yield x, rgb_gen.classes[idx: idx+rgb_gen.batch_size]

train_gen = gen_func(train_split_df, train_datagen, batch_size=batch_size)
valid_gen = gen_func(valid_split_df, val_datagen, batch_size=batch_size)

In [32]:
def fmeasure(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall))

We're going to train our model with SGD and very low learning rate

In [33]:
early_stp = EarlyStopping(monitor="val_loss", mode='min', patience=5)
model_ckpt = ModelCheckpoint('resnet_1_layer.h5', save_best_only=True, mode='min', monitor='val_loss', verbose=1, save_weights_only=True)

# opt = optimizers.SGD(lr=0.001, decay = 1e-6, momentum = 0.9, nesterov = True)
opt = optimizers.Adam(lr=1e-4)
model.compile(opt, loss = 'binary_crossentropy', metrics=['accuracy', fmeasure])
model.reset_states()

In [ ]:
model.fit_generator(train_gen,
                    steps_per_epoch=train_split_df.shape[0] / batch_size, epochs=100,
                    validation_data=valid_gen, 
                    validation_steps = valid_split_df.shape[0]/batch_size, 
                    callbacks=[early_stp, model_ckpt], 
                    workers = 10, 
                    max_queue_size=20)

## Ignore next message from keras, values are replaced anyways
## Ignore next message from keras, values are replaced anyways
Epoch 1/100
Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.
Reinserting dataframe: 3000 images
Reinserting dataframe: 9600 images
 68/150 [============>.................] - ETA: 3:06 - loss: 0.5902 - acc: 0.6920 - fmeasure: 0.5718

In [ ]:
model = get_model()

#train only last 20 layer
for layer in model.layers:
    layer.trainable = True

opt = optimizers.Adam(lr=1e-5)
# opt = optimizers.SGD(lr=0.001, decay = 1e-6, momentum = 0.9, nesterov = True)
model.compile(opt, loss = 'binary_crossentropy', metrics=['accuracy', fmeasure])
    
model.summary()

In [ ]:
early_stp = EarlyStopping(monitor="val_loss", mode='min', patience=5)
model_ckpt = ModelCheckpoint('resnet_all_layers.h5', save_best_only=True, mode='min', monitor='val_loss', verbose=1, save_weights_only=True)

model.load_weights('resnet_1_layer.h5')


In [ ]:
batch_size = 32
model.fit_generator(train_datagen.flow(X_tra, y_tra, batch_size=batch_size),                     
                    steps_per_epoch=len(X_tra) / batch_size, epochs=100,
                    validation_data=val_datagen.flow(X_val, y_val, batch_size=batch_size), 
                    validation_steps = len(X_val)/batch_size, callbacks=[early_stp, model_ckpt], workers = 10, max_queue_size=20)

In [ ]:
test_imgs = get_imgs(test_data_dir, test)

In [ ]:
test_datagen.fit(test_imgs)

In [ ]:
pred = model.predict_generator(test_datagen.flow(test_imgs, batch_size=512, shuffle=False), verbose=1, workers=8)

In [ ]:
sub = pd.read_csv('./data/sample_submission.csv')
sub.iloc[:, 1:] = pred.round().astype(int)
sub.head()

In [ ]:
sub.to_csv('submission.csv', index=False)

In [ ]:
sub.shape

## Final Thoughts

This submission should get you around $\approx$0.80 on the LB. and if you've noticed that our last epochs val_fmeasure is the same so it means that our val set is represantion of the test set and you can train for many epochs with EarlyStopping without worring about overfitting on val or train set.

### How to improve from here?
You can change many things which will let you get higher LB score. Following is a small list
<ul>
    <li> Change the Image size to bigger number </li>
    <li> Increse the number of epoch in the fully trainable network($2^{nd}$ training) </li>
    <li> Use diffrent architechure. you'll get more info on that [here](keras.io/applications/)</li>
    <li> If nothing works ensemble is your best friend </li>
</ul>

I'll try to keep improving this notebook. Feel free to contribuite.

Thanks
